# **Title: Pare AI Tool-Calling Chatbot**

**1️⃣ Imports & Setup - (API, model config)**

In [8]:
import os
import json
import random
from typing import Dict
import google.generativeai as genai
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

# Paste your Gemini API key here
API_KEY = os.getenv("GEMINI_API_KEY")

if not API_KEY:
    raise ValueError("GEMINI_API_KEY environment variable not set")

genai.configure(api_key=API_KEY)

model = genai.GenerativeModel("models/gemini-2.5-flash")

appointments: Dict[str, Dict] = {}


 **2️⃣ Backend Tool Functions**

In [9]:
def generate_appointment_id() -> str:
    return f"APT-{random.randint(1000, 9999)}"


def book_appointment(name: str, address: str, date: str, time: str) -> dict:
    appointment_id = generate_appointment_id()

    appointments[appointment_id] = {
        "name": name,
        "address": address,
        "date": date,
        "time": time
    }

    return {
        "status": "confirmed",
        "appointment_id": appointment_id,
        "name": name,
        "address": address,
        "date": date,
        "time": time
    }


def cancel_appointment(appointment_id: str) -> dict:
    if appointment_id not in appointments:
        return {"status": "not_found", "appointment_id": appointment_id}

    del appointments[appointment_id]
    return {"status": "cancelled", "appointment_id": appointment_id}


def print_appointments():
    print("\n📅 CURRENT APPOINTMENTS")

    if not appointments:
        print("No appointments booked.")
        return

    for k, v in appointments.items():
        print(k, "→", v)


**3️⃣ System Prompt**

In [10]:
SYSTEM_PROMPT = """
You are a professional AI assistant for Pare, a company specializing in premium polymer wall and ceiling products.

STRICT RULES:

1. Product Knowledge:
You must answer ONLY using the Pare Product Knowledge Base below.
If a user asks anything outside this KB, reply politely that the information is not available.

2. Language:
Always respond in the same language as the user (English or Hindi).

3. Appointment Booking Flow:
If the user wants to book an appointment, ensure you collect ALL required fields:
- name
- address
- date
- time

If any field is missing, ask only for the missing information.

4. Date & Time Normalization:
- Convert relative dates like "tomorrow" into YYYY-MM-DD format.
- Convert time into 24-hour HH:MM format.

5. Tool Usage:
When all appointment details are available, call the appropriate tool using strict JSON.

6. Output Format (MANDATORY):
You must always output valid JSON only — no extra text.

Normal response:
{"type": "message", "content": "..."}

Tool call:
{"type": "tool_call", "name": "book_appointment", "args": {...}}

Cancel:
{"type": "tool_call", "name": "cancel_appointment", "args": {...}}

====================
Pare Product Knowledge Base:

INNOV+ (Interiors): Wood-look soffits & louvers, tongue-and-groove, waterproof, termite-proof.

INNOV2+ (3D Interiors): Wave, Dome, Stripes, Prism feature walls.

DURA+ (Exteriors): UV-protected, weatherproof cladding; variants Norma, Stretta.

EASY+ (Quick Install): Direct installation without plywood backing.

LUXE (Luxury): Dead-matt finish, anti-bacterial; variants Panelo, Flut, Verto.

Acoustic: Louvers designed specifically for sound absorption.
====================
"""


**4️⃣ LLM Router / Chat Function**

In [11]:
def clean_json(text: str) -> str:
    text = text.strip()
    if text.startswith("```"):
        text = text.replace("```json", "").replace("```", "").strip()
    return text


def process_llm_output(raw_text: str):
    try:
        cleaned = clean_json(raw_text)
        data = json.loads(cleaned)

        if data["type"] == "message":
            print("🤖 BOT:", data["content"])

        elif data["type"] == "tool_call":
            tool = data["name"]
            args = data["args"]

            if tool == "book_appointment":
                result = book_appointment(**args)

            elif tool == "cancel_appointment":
                result = cancel_appointment(**args)

            print("🛠 TOOL RESULT:", result)

    except Exception as e:
        print("⚠️ Parsing Error:", e)
        print("Raw output:", raw_text)


def chat(user_input: str):
    try:
        response = model.generate_content(
            SYSTEM_PROMPT + "\nUser: " + user_input
        )

        output = response.text.strip()
        process_llm_output(output)

    except Exception as e:
        print("⚠️ Gemini Error:", e)


**5️⃣ Vision Chat Function**

In [12]:
def chat_with_image(user_text: str, image_path: str):
    try:
        img = Image.open(image_path)

        response = model.generate_content(
            [SYSTEM_PROMPT, user_text, img]
        )

        output = response.text.strip()
        process_llm_output(output)

    except Exception as e:
        print("⚠️ Vision Error:", e)


**6️⃣ Demo 1 (English Q&A + Booking)**

In [13]:
print_appointments()

chat("Tell me about DURA+ exteriors.")

chat("Book an appointment for tomorrow at 4 pm. Name: Vedant Parikh. Address: Andheri West, Mumbai.")

print_appointments()



📅 CURRENT APPOINTMENTS
No appointments booked.
🤖 BOT: DURA+ exteriors offer UV-protected, weatherproof cladding, and are available in Norma and Stretta variants.
🛠 TOOL RESULT: {'status': 'confirmed', 'appointment_id': 'APT-1545', 'name': 'Vedant Parikh', 'address': 'Andheri West, Mumbai', 'date': '2024-05-28', 'time': '16:00'}

📅 CURRENT APPOINTMENTS
APT-1545 → {'name': 'Vedant Parikh', 'address': 'Andheri West, Mumbai', 'date': '2024-05-28', 'time': '16:00'}


**7️⃣ Demo 2 (Cancellation)**

In [14]:
print_appointments()

chat("Cancel my appointment APT-1545")  # replace real ID

print_appointments()



📅 CURRENT APPOINTMENTS
APT-1545 → {'name': 'Vedant Parikh', 'address': 'Andheri West, Mumbai', 'date': '2024-05-28', 'time': '16:00'}
🛠 TOOL RESULT: {'status': 'cancelled', 'appointment_id': 'APT-1545'}

📅 CURRENT APPOINTMENTS
No appointments booked.


**8️⃣ Demo 3 (Hindi Interaction)**

In [15]:
print_appointments()

chat("INNOV+ interiors के बारे में बताइए।")

chat("मुझे कल शाम 5 बजे की अपॉइंटमेंट बुक करनी है। नाम: वेदांत, पता: चर्चगेट।")

# chat("तारीख: 2026-02-09, समय: 17:00")


print_appointments()



📅 CURRENT APPOINTMENTS
No appointments booked.
🤖 BOT: INNOV+ (इंटीरियर्स): इसमें लकड़ी जैसे दिखने वाले सोफिट और लूवर, टंग-एंड-ग्रूव डिज़ाइन, वाटरप्रूफ और दीमक-प्रूफ उत्पाद शामिल हैं।
🛠 TOOL RESULT: {'status': 'confirmed', 'appointment_id': 'APT-7947', 'name': 'वेदांत', 'address': 'चर्चगेट', 'date': '2024-05-16', 'time': '17:00'}

📅 CURRENT APPOINTMENTS
APT-7947 → {'name': 'वेदांत', 'address': 'चर्चगेट', 'date': '2024-05-16', 'time': '17:00'}


**9️⃣ Demo 4 (Business Card)**

In [16]:
chat_with_image(
    "I want to book an appointment for tomorrow at 11 am. I'm sharing my business card.",
    "Business_Card.png"
)

print_appointments()


🛠 TOOL RESULT: {'status': 'confirmed', 'appointment_id': 'APT-7463', 'name': 'Amit Shah', 'address': '502, Alpha Plaza, Andheri West, Mumbai', 'date': '2024-05-16', 'time': '11:00'}

📅 CURRENT APPOINTMENTS
APT-7947 → {'name': 'वेदांत', 'address': 'चर्चगेट', 'date': '2024-05-16', 'time': '17:00'}
APT-7463 → {'name': 'Amit Shah', 'address': '502, Alpha Plaza, Andheri West, Mumbai', 'date': '2024-05-16', 'time': '11:00'}
